<h1 align=center><font size = 5>Loading Canada Geospatial Data</font></h1>

### Code to Extract Data from Web http://cocl.us/Geospatial_data

In [1]:
import pandas as pd

#Download file
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

#Reading data from file
dfgeo=pd.read_csv('Geospatial_Coordinates.csv', sep=',', float_precision=None)
dfgeo.rename(columns={'Postal Code': 'Postcode'}, inplace=True)
dfgeo['Postcode'] = dfgeo['Postcode'].str.strip()
dfgeo.head(10)

Data downloaded!


,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


### Reading Canada Neighborhood data which was extracted from Wiki

In [2]:
#Reading Canada Neighborhood data that was scraped from Web
import pandas as pd
import numpy as np
df=pd.read_csv('Canada_Neighborhood.txt', sep=",", names=["Postcode", "Borough", "Neighbourhood"],usecols=["Postcode", "Borough", "Neighbourhood"], header=0)

#striping unwanted spaces.
df.rename(columns=lambda x: x.strip())
df['Postcode'] = df['Postcode'].str.strip()
df['Borough'] = df['Borough'].str.strip()
df['Neighbourhood'] = df['Neighbourhood'].str.strip()

#deleting Not assigned value from Borough
df = df[df.Borough != 'Not assigned']

#replacing Not assigned value of Neighbourhood with Borough column value
df['Neighbourhood'] = np.where(df['Neighbourhood'] == "Not assigned", df['Borough'], df['Neighbourhood'])

#Concatenating same Neighbourhood values group by Postcode
df['Neighbourhood']= df[['Postcode','Borough','Neighbourhood']].groupby(['Postcode'])['Neighbourhood'].transform(lambda x: ', '.join(x))
df=df[['Postcode','Borough','Neighbourhood']].drop_duplicates(inplace=False)
df.reset_index(drop=True, inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [3]:
# This is to update Canada Neighborhood dataframe with Canada Geospatial data(Latitude and Longitude)
vdic = pd.Series(dfgeo.Latitude.values, index=dfgeo.Postcode).to_dict()
df.loc[df.Postcode.isin(vdic.keys()), 'Latitude'] = df.loc[df.Postcode.isin(vdic.keys()), 'Postcode'].map(vdic)
vdic = pd.Series(dfgeo.Longitude.values, index=dfgeo.Postcode).to_dict()
df.loc[df.Postcode.isin(vdic.keys()), 'Longitude'] = df.loc[df.Postcode.isin(vdic.keys()), 'Postcode'].map(vdic)

df.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
